<a href="https://colab.research.google.com/github/NikolayKuraga/ML_lab_2/blob/master/2_nn_text_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers -q

In [2]:
!echo 'WOW! I can execute shell commands from Jupyter Notebook!'
!echo 'Interesting, what shell is used under Windows...'
!echo "Current shell is: \"${SHELL}\"."
!pip install fast_langdetect iso-639 nltk pandas >/dev/null


import enum
import string
import typing as t


import fast_langdetect
import iso639
import nltk.corpus
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import gensim
from sentence_transformers import SentenceTransformer

nltk.download('stopwords')
print('Done')

WOW! I can execute shell commands from Jupyter Notebook!
Interesting, what shell is used under Windows...
Current shell is: "/bin/bash".
Done


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df_trn = pd.read_csv('https://raw.githubusercontent.com/NikolayKuraga/ML_lab_2/refs/heads/master/train.csv')
df_tst = pd.read_csv('https://raw.githubusercontent.com/NikolayKuraga/ML_lab_2/refs/heads/master/test.csv')

print('Head of train dataset:')
print(df_trn.head())
print()
print('Info of train dataset')
print(df_trn.info())

Head of train dataset:
   Class Index                                              Title  \
0            3  Wall St. Bears Claw Back Into the Black (Reuters)   
1            3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2            3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3            3  Iraq Halts Oil Exports from Main Southern Pipe...   
4            3  Oil prices soar to all-time record, posing new...   

                                         Description  
0  Reuters - Short-sellers, Wall Street's dwindli...  
1  Reuters - Private investment firm Carlyle Grou...  
2  Reuters - Soaring crude prices plus worries\ab...  
3  Reuters - Authorities have halted oil export\f...  
4  AFP - Tearaway world oil prices, toppling reco...  

Info of train dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Class Inde

### 1. Try some text cleaning techniques from the list below (at least 2). Does any of it improved your model quality? Try to assume why.

In [4]:
# 1. Try some text cleaning techniques from the list below (at least 2). Does any of it
# improved your model quality? Try to assume why.
#     a. stop words removing
#     b. punctuation removing
#     c. Trash (Extra spaces / special symbols (like @#~<> etc)) removing
#     d. digits removing


# First of all tried to detect what languages are used.
# Found two ways:
#     1) langdetect -- "traditional" module, really slow (several seconds for 1000 entities).
#     2) fast_langdetect -- fast "zoomer" language-machine-learning based module.
# I will use fast_langdetect model 'cause I'm zoomer.

def detectLanguage(text: str) -> str:
    code = fast_langdetect.detect_language(text).lower()
    name = iso639.languages.get(alpha2=code).name
    name = ''.join(name.split()).lower()
    return name


df_trn['Lang'] = (df_trn['Title'] + ' ' + df_trn['Description']).apply(detectLanguage)
df_tst['Lang'] = (df_tst['Title'] + ' ' + df_tst['Description']).apply(detectLanguage)

print('Train dataset')
print(df_trn['Lang'].value_counts())
print()
print('Test dataset')
print(df_tst['Lang'].value_counts())

Train dataset
Lang
english           119944
french                21
spanish               10
polish                 9
german                 7
italian                5
westernfrisian         3
ukrainian              1
Name: count, dtype: int64

Test dataset
Lang
english    7596
polish        3
french        1
Name: count, dtype: int64


In [5]:
# Find necessary stopwords in nltk database.
stopwords = set()
for lang in df_trn['Lang'].unique():
    try:
        stopwords |= set(nltk.corpus.stopwords.words(lang))
    except:
        print(f'no "{lang}" language in stopwords database?')

print()
print(f'Got so many ({len(stopwords)}) stopwords from different languages!')

def cleanTextFromStopwords8Punc(txt: str) -> str:
    txt = ''.join([sym for sym in txt if sym not in string.punctuation])
    txt = ' '.join([wrd for wrd in txt.lower().split() if wrd not in stopwords])
    return txt

def cleanTextFromStopwords8Punc8Dig(txt: str) -> str:
    txt = ''.join([i for i in txt if not i.isdigit()])
    txt = ''.join([sym for sym in txt if sym not in string.punctuation])
    txt = ' '.join([wrd for wrd in txt.lower().split() if wrd not in stopwords])
    return txt

df_trn['Title no stopwords8punc'] = df_trn['Title'].apply(cleanTextFromStopwords8Punc)
df_trn['Title no stopwords8punc8dig'] = df_trn['Title'].apply(cleanTextFromStopwords8Punc8Dig) #Title no stopwords8punc8dig
df_trn['Description no stopwords8punc'] = df_trn['Description'].apply(cleanTextFromStopwords8Punc) #Description no stopwords8punc
df_trn['Description no stopwords8punc8dig'] = df_trn['Description'].apply(cleanTextFromStopwords8Punc8Dig)

df_tst['Title no stopwords8punc'] = df_tst['Title'].apply(cleanTextFromStopwords8Punc)
df_tst['Title no stopwords8punc8dig'] = df_tst['Title'].apply(cleanTextFromStopwords8Punc8Dig)
df_tst['Description no stopwords8punc'] = df_tst['Description'].apply(cleanTextFromStopwords8Punc)
df_tst['Description no stopwords8punc8dig'] = df_tst['Description'].apply(cleanTextFromStopwords8Punc8Dig)

print()
print(df_trn['Title'].iloc[35:40])
print()
print(df_trn['Title no stopwords8punc'].iloc[35:40])
print()
print(df_trn['Title no stopwords8punc8dig'].iloc[35:40])

no "polish" language in stopwords database?
no "ukrainian" language in stopwords database?
no "westernfrisian" language in stopwords database?

Got so many (1089) stopwords from different languages!

35                                  Steady as they go
36         Google IPO: Type in 'confusing,' 'secrecy'
37                        A bargain hunter's paradise
38     Researchers seek to untangle the e-mail thread
39    Microsoft Corp. 2.0: a kinder corporate culture
Name: Title, dtype: object

35                                     steady go
36             google ipo type confusing secrecy
37                      bargain hunters paradise
38        researchers seek untangle email thread
39    microsoft corp 20 kinder corporate culture
Name: Title no stopwords8punc, dtype: object

35                                  steady go
36          google ipo type confusing secrecy
37                   bargain hunters paradise
38     researchers seek untangle email thread
39    microsoft corp kinder

### 2. Try to apply stemming and lemmatization.

In [6]:
X_train =  df_trn[['Title no stopwords8punc8dig','Description no stopwords8punc8dig']]
y_train =  df_trn['Class Index']

X_test =  df_tst[['Title no stopwords8punc8dig','Description no stopwords8punc8dig']]

In [7]:
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
def lemmatzation(lst):
   new_lem = []
   for i in lst:
       i = lemmatizer.lemmatize(i)
       new_lem.append(i)
   return new_lem

train_x_lem = X_train.apply(lemmatzation)
test_x_lem = X_test.apply(lemmatzation)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
def lemmatize_text(txt):
    lemmatized_words = [lemmatizer.lemmatize(w) for w in txt.split()]  # Лемматизация
    return lemmatized_words


# Применим лемматизацию к каждому столбцу в тест и трейн
train_x_lem = pd.DataFrame()
train_x_lem['title'] = X_train['Title no stopwords8punc8dig'].apply(lemmatize_text)
train_x_lem['description'] = X_train['Description no stopwords8punc8dig'].apply(lemmatize_text)

test_x_lem = pd.DataFrame()
test_x_lem['title'] = X_test['Title no stopwords8punc8dig'].apply(lemmatize_text)
test_x_lem['description'] = X_test['Description no stopwords8punc8dig'].apply(lemmatize_text)

In [9]:
stemmer = SnowballStemmer("english")

def stemming(txt):
    stem_words = [stemmer.stem(w) for w in txt.split()]
    return stem_words # ' '.join(stem_words)

# Применим стемминг к каждому столбцу в тест и трейн
train_x_stem = pd.DataFrame()
train_x_stem['title'] = X_train['Title no stopwords8punc8dig'].apply(stemming)
train_x_stem['description'] = X_train['Description no stopwords8punc8dig'].apply(stemming)

test_x_stem = pd.DataFrame()
test_x_stem['title'] = X_test['Title no stopwords8punc8dig'].apply(stemming)
test_x_stem['description'] = X_test['Description no stopwords8punc8dig'].apply(stemming)

In [10]:
# Итого два варианта: train_x_lem, test_x_lem и train_x_stem,test_x_stem
# Which is improve model quality better узнаем только по модели, поэтому дальше используем оба набора данных

In [11]:
print(train_x_lem.head(3))
print(train_x_stem.head(3))

                                               title  \
0       [wall, st, bear, claw, back, black, reuters]   
1  [carlyle, look, toward, commercial, aerospace,...   
2     [oil, economy, cloud, stock, outlook, reuters]   

                                         description  
0  [reuters, shortsellers, wall, street, dwindlin...  
1  [reuters, private, investment, firm, carlyle, ...  
2  [reuters, soaring, crude, price, plus, worries...  
                                               title  \
0        [wall, st, bear, claw, back, black, reuter]   
1  [carlyl, look, toward, commerci, aerospac, reu...   
2      [oil, economi, cloud, stock, outlook, reuter]   

                                         description  
0  [reuter, shortsel, wall, street, dwindlingband...  
1  [reuter, privat, invest, firm, carlyl, groupwh...  
2  [reuter, soar, crude, price, plus, worriesabou...  


In [12]:
tfidf_vectorizer = TfidfVectorizer()

In [23]:
title_tfidf_lem = tfidf_vectorizer.fit_transform(train_x_lem['title'])
desc_tfidf_lem = tfidf_vectorizer.fit_transform(train_x_lem['description'])
title_test_tfidf_lem = tfidf_vectorizer.transform(test_x_lem['title'])
desc_test_tfidf_lem = tfidf_vectorizer.transform(test_x_lem['description'])


In [ ]:
title_tfidf_stem = tfidf_vectorizer.fit_transform(train_x_stem['title'])
desc_tfidf_stem = tfidf_vectorizer.fit_transform(train_x_stem['description'])
title_test_tfidf_stem = tfidf_vectorizer.transform(test_x_stem['title'])
desc_test_tfidf_stem = tfidf_vectorizer.transform(test_x_stem['description'])

In [ ]:
word2vec_title_lem = gensim.models.Word2Vec(train_x_lem['title'], min_count=1,
								vector_size=100, window=5)
word2vec_desc_lem = gensim.models.Word2Vec(train_x_lem['description'], min_count=1,
								vector_size=100, window=5)
word2vec_test_title_lem = gensim.models.Word2Vec(test_x_lem['title'], min_count=1,
								vector_size=100, window=5)
word2vec_test_desc_lem = gensim.models.Word2Vec(test_x_lem['description'], min_count=1,
								vector_size=100, window=5)

In [26]:
word2vec_title_stem = gensim.models.Word2Vec(train_x_stem['title'], min_count=1,
								vector_size=100, window=5)
word2vec_desc_stem = gensim.models.Word2Vec(train_x_stem['description'], min_count=1,
								vector_size=100, window=5)
word2vec_test_title_stem = gensim.models.Word2Vec(test_x_stem['title'], min_count=1,
								vector_size=100, window=5)
word2vec_test_desc_stem = gensim.models.Word2Vec(test_x_stem['description'], min_count=1,
								vector_size=100, window=5)

In [ ]:
transformer = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
vect_transformers_title_lem = transformer.encode(train_x_lem['title'])
vect_transformers_desc_lem = transformer.encode(train_x_lem['description'])
vect_transformers_test_title_lem = transformer.encode(test_x_lem['title'])
vect_transformers_test_desc_lem = transformer.encode(test_x_lem['description'])

In [ ]:
vect_transformers_title_stem = transformer.encode(train_x_stem['title'])
vect_transformers_desc_stem = transformer.encode(train_x_stem['description'])
vect_transformers_test_title_stem = transformer.encode(test_x_stem['title'])
vect_transformers_test_desc_stem = transformer.encode(test_x_stem['description'])